In [1]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from random import randint, uniform
from math import pi
import qutip

#Inputs
n_layers = 2
n_wires = 4
n_targets = 10
iteracoes = 2


def RandomLayers(n_layers,n_wires,n_targets):

    #Criação do vetor de matrizes-peso
    weights = []
    for i in range(n_layers):
        row = []
        for j in range(n_wires):
            column = []
            for k in range(n_wires):   #Matriz quadrada (podemos alterar isso)
                column.append(uniform(0,2*pi))
            row.append(column)
        weights.append(row)

    for i in range(n_layers):
        print("Layer: {}".format(i))
        print("Matriz de pesos: {}".format(weights[i]))
        print("\n")


    #Seed aleatório (template dentro do RandomLayers)
    seed = randint(1, 9999)

    #Cria alvos do cálculo de expressabilidade
    def normaliza(v):
        v = v / (np.linalg.norm(v))
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states


    #Cria o circuito
    @qml.qnode(dev)
    def circuit(weights, seed=None):
        for i in range(n_layers):
            qml.RandomLayers(weights=weights[i], wires=range(n_wires), seed=seed)
        return qml.state()
    

    #Armazenamos na variável amplt as probabilidades do nosso ket resultado (tensor do numpy) para que possamos usar a função fidelity
    #Para tanto, tiramos o produto interno dele com ele mesmo e calculamos sua norma. Esta função nos retorna um vetor (não o tipo de dados usual tensor de rank 1)
    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        aux = np.array(aux)    
        return aux
    


    #Instanciações
    print(qml.draw(circuit, expansion_strategy="device")(weights, seed))
    print("\nCircuito correspondente ao seed: {}".format(seed))
    ket = circuit(weights, seed)
    amplt = ket_in_amplt(ket)
    print("\nEstado resultado do circuito: ", ket)
    print("\nAmplitudes", amplt)
    target_states = cria_alvos(n_targets, n_wires)



    
    #Expressabilidade
    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity
    

    #Grau de Emaranhamento
    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
            rho_k_sq = ket.ptrace([k])**2
            entanglement_sum += rho_k_sq.tr()
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q



    Q = compute_Q_ptrace(ket, n_wires)
    #print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    #print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]


    return k


dev = qml.device('default.qubit', wires=n_wires)


k=[]
for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(RandomLayers(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()



= Modelo: 1 =

Layer: 0
Matriz de pesos: [[3.618509409726296, 3.4592164069602394, 2.2580424204556793, 1.1495110173586538], [1.015196401014578, 5.891103826984866, 3.1480445063251214, 0.819927625979154], [4.917180053921045, 3.1070205808199534, 3.7762284910905977, 1.0043740518667623], [5.005144007206317, 3.009849644260168, 0.9676055455230593, 4.7588005674268254]]


Layer: 1
Matriz de pesos: [[5.119983830944609, 4.704458444429578, 5.046836234066237, 2.9243129306298528], [2.7610068617942405, 3.3516817324346837, 3.58848542106485, 3.2172361893706487], [2.3294066608152617, 1.764162123954179, 2.913583621801844, 3.756349269957744], [5.746400676435534, 6.000212986559956, 1.6432439393863572, 2.7348929497983514]]


0: ──RZ(5.89)────────────────────────────────────────────╭●─╭●──RZ(3.11)─╭X──RY(5.01)──RZ(3.35)───
1: ──RZ(3.62)─╭X──RY(1.02)──RY(0.82)─────────────────────│──╰X───────────╰●─╭●─────────RZ(5.12)───
2: ───────────│──╭X─────────RY(4.92)─────────────────────│──╭X──────────────╰X─────────RY

,Grau de Emaranhamento,Expressabilidade
count,2.000000,2.000000
mean,0.435862,0.045062
std,0.009269,0.005195
min,0.429308,0.041389
25%,0.432585,0.043225
50%,0.435862,0.045062
75%,0.439139,0.046899
max,0.442416,0.048735
